## Note: Use GPU as your runtime Hardware accelerator

### How to do that?

> 1. Click on Runtime
> 2. Select Change Runtime type
> 3. Choose T4 GPU as your Hardware accelerator

In [1]:
!pip install transformers ctransformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.7 MB/s eta 0:00:00


In [7]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.0 MB/s eta 0:00:00


In [1]:
import torch

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## StableLM

StableLM-Base-Alpha-3B-v2 is a 3 billion parameter decoder-only language model pre-trained on diverse English datasets. This model is the successor to the first StableLM-Base-Alpha-3B model, addressing previous shortcomings through the use of improved data sources and mixture ratios.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_name = "stabilityai/stablelm-base-alpha-3b-v2"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
  model_name,
  trust_remote_code=True,
  torch_dtype="auto",
)

A new version of the following files was downloaded from https://huggingface.co/stabilityai/stablelm-base-alpha-3b-v2:
- configuration_stablelm_alpha.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/stabilityai/stablelm-base-alpha-3b-v2:
- modeling_stablelm_alpha.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model.cuda()

StableLMAlphaForCausalLM(
  (transformer): StableLMAlphaModel(
    (embed): Embedding(50432, 2560)
    (layers): ModuleList(
      (0-31): 32 x DecoderLayer(
        (norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (attention): Attention(
          (qkv_proj): Linear(in_features=2560, out_features=7680, bias=False)
          (out_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): MLP(
          (gate_proj): Linear(in_features=2560, out_features=13824, bias=False)
          (out_proj): Linear(in_features=6912, out_features=2560, bias=False)
          (act): SiLU()
        )
      )
    )
    (final_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=2560, out_features=50432, bias=False)
)

In [ ]:
inputs = tokenizer("The weather is always wonderful", return_tensors="pt").to(device)

In [ ]:
tokens = model.generate(
  **inputs,
  max_new_tokens=300,
  temperature=0.75,
  top_p=0.95,
  do_sample=True,
)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [ ]:
print(tokenizer.decode(tokens[0], skip_special_tokens=True))

The weather is always wonderful in the mountains, and the views are incredible. If you're visiting this area, I highly recommend you make time for a short trip to the mountains.
- The B&B is located in a small town in the mountains of the foothills of the Alps. It's about an hour's drive from the airport in Turin.
- If you're driving, I highly recommend taking a scenic route to avoid traffic. The drive is beautiful and takes you through some of the most beautiful landscapes in the world.
- The Alps are famous for their snow, and there are several ski resorts in the area. If you're interested in skiing, this is the perfect place to go.
- The region is also famous for its cheese and wine, so if you're a foodie, you'll definitely want to stop by one of the local cheese and wine shops.
- There are several hiking trails in the area, so if you're into hiking, you'll definitely want to check them out.
- If you're a nature lover, the Alps are the perfect place for you. There are so many beauti

In [4]:
torch.cuda.empty_cache()

## Orca-mini-3b

In [3]:
from transformers import LlamaForCausalLM, LlamaTokenizer

In [4]:
model_path = 'psmathur/orca_mini_3b'
tokenizer = LlamaTokenizer.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map='auto',
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
prompt = "Write about ISRO and his achievements"

In [6]:
tokens = tokenizer.encode(prompt)
tokens = torch.LongTensor(tokens).unsqueeze(0)
tokens = tokens.to(device)

In [9]:
instance = {'input_ids': tokens,'top_p': 1.0, 'temperature':0.2, 'generate_len': 512, 'top_k': 30}

In [10]:
length = len(tokens[0])
with torch.no_grad():
  response = model.generate(
              input_ids=tokens,
              max_length=length+instance['generate_len'],
              top_p=instance['top_p'],
              temperature=instance['temperature'],
              top_k=instance['top_k'],
              use_cache=True,
              do_sample=True,
        )

In [11]:
response

tensor([[    1, 14734,   562,  3461,  2784,   291,   492, 14013, 31843,    13,
          5468,   620,  5867,  2858,  9921,   352,  1921,  2784, 31861,   322,
           260,  2138,  4938,   287,  2600,   342,   393,  3856,   288, 31822,
         31853, 31877, 31887, 31877, 31843,   571,   322,  4181,   329, 15021,
         31844, 14447, 31844,   291, 13317,  1639,   328,  2470,  2138, 13774,
         31843,  3461,  2784,   470,   985,  2259,  7872,   289,   266,  1915,
           287,  2138, 11317,   291,   470,  7222,   869,  1677, 26286,   288,
           266,  1249, 31843,    13,  9040,   287,   266,   758, 12187, 14013,
           287,  3461,  2784,   322,   266,  4540,   287,   266,  3162,  1716,
         27082,   352, 19876,  1515, 31861,  1934,   287, 22688, 31843,  1872,
         22688,   397,  3213,   289,  1673,  5121,   291,  1160,  1705,   289,
           266,  3162,  1444,   291,  2309,  4343, 31843, 23647,  1515, 31854,
         31853, 31849, 31844,  5355,   288, 31822, 3

In [12]:
output = response[0][length:]
output

tensor([31843,    13,  5468,   620,  5867,  2858,  9921,   352,  1921,  2784,
        31861,   322,   260,  2138,  4938,   287,  2600,   342,   393,  3856,
          288, 31822, 31853, 31877, 31887, 31877, 31843,   571,   322,  4181,
          329, 15021, 31844, 14447, 31844,   291, 13317,  1639,   328,  2470,
         2138, 13774, 31843,  3461,  2784,   470,   985,  2259,  7872,   289,
          266,  1915,   287,  2138, 11317,   291,   470,  7222,   869,  1677,
        26286,   288,   266,  1249, 31843,    13,  9040,   287,   266,   758,
        12187, 14013,   287,  3461,  2784,   322,   266,  4540,   287,   266,
         3162,  1716, 27082,   352, 19876,  1515, 31861,  1934,   287, 22688,
        31843,  1872, 22688,   397,  3213,   289,  1673,  5121,   291,  1160,
         1705,   289,   266,  3162,  1444,   291,  2309,  4343, 31843, 23647,
         1515, 31854, 31853, 31849, 31844,  5355,   288, 31822, 31853, 31877,
        31886, 31853, 31844,   393,   266,   712, 10890,  5355, 

In [13]:
result = tokenizer.decode(output, skip_special_tokens=True)

In [14]:
result

'.\nIndian Space Research Organization (ISRO) is a space agency of India that was established in 1969. It is responsible for designing, launching, and conducting research on various space missions. ISRO has made significant contributions to the field of space exploration and has achieved many milestones in the process.\nOne of the most notable achievements of ISRO is the launch of the Indian National Satellite (INSAT) series of satellites. These satellites are designed to provide communication and data services to the Indian government and private organizations. INSAT-1A, launched in 1981, was the first satellite launched by ISRO and was followed by INSAT-1B, INSAT-1C, INSAT-1D, INSAT-1E, INSAT-1F, INSAT-2A, INSAT-2B, INSAT-2C, INSAT-2D, INSAT-3A, INSAT-3B, INSAT-3C, INSAT-3D, INSAT-4A, INSAT-4B, INSAT-4C, INSAT-4D, INSAT-4E, INSAT-4F, INSAT-5A, INSAT-5B, INSAT-5C, INSAT-5D, INSAT-6A, INSAT-6B, INSAT-6C, INSAT-6D, INSAT-6E, INSAT-6F, INSAT-7A, INSAT-7B, INSAT-7C, INSAT-7D, INSAT-8A, IN

## GPT4all

GPT4All is an ecosystem to train and deploy powerful and customized large language models that run locally on consumer grade CPUs.

The goal is simple - be the best instruction tuned assistant-style language model that any person or enterprise can freely use, distribute and build on.

A GPT4All model is a 3GB - 8GB file that you can download and plug into the GPT4All open-source ecosystem software. Nomic AI supports and maintains this software ecosystem to enforce quality and security alongside spearheading the effort to allow any person or enterprise to easily train and deploy their own on-edge large language models.

In [ ]:
!pip install gpt4all==0.2.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: gpt4all
    Found existing installation: gpt4all 1.0.9
    Uninstalling gpt4all-1.0.9:
      Successfully uninstalled gpt4all-1.0.9


In [15]:
torch.cuda.empty_cache()

## Note: In the below code, if you find errors, Restart the runtime and run from import gpt4all freshly

In [ ]:
import gpt4all

### Check all the available models on GPT4all using this link: [GPT4All](https://gpt4all.io/index.html)

In [ ]:
gpt4_model = gpt4all.GPT4All(model_name="ggml-gpt4all-j-v1.2-jazzy")

Found model file.


## Predictions using GPT4all

In [ ]:
output = gpt4_model.generate("Write a python program to add two variables and print the sum")

Here's a Python program that adds two variables and prints the sum:```pythonnum1 = 10  # first variable to addnum2 = 5  # second variable to addsum_of_nums = num1 + num2  # sum of the two variablesprint(sum_of_nums)  # print the sum```


In [ ]:
output

"Here's a Python program that adds two variables and prints the sum:```pythonnum1 = 10  # first variable to addnum2 = 5  # second variable to addsum_of_nums = num1 + num2  # sum of the two variablesprint(sum_of_nums)  # print the sum```"

In [ ]:
torch.cuda.empty_cache()